# RNA Reactivity Prediction
https://www.kaggle.com/competitions/stanford-ribonanza-rna-folding/overview

In [1]:
TEST_DATA_PATH = 'E:/data/Ribonanza_RNA_folding/test_sequences.csv'
TEST_DATA_EXT_PATH = 'E:/data/Ribonanza_RNA_folding/test_extracted.csv'
BPP_DATA_PATH = 'E:/data/Ribonanza_RNA_folding/Ribonanza_bpp_files/extra_data'
SUBMISSION_SAMPLE_FILE_PATH = 'E:/data/Ribonanza_RNA_folding/sample_submission.csv'
SUBMISSION_FILE_PATH = 'E:/data/Ribonanza_RNA_folding/submission.csv'
VOCAB_PATH = 'E:/data/Ribonanza_RNA_folding/vocab.csv'
DATA_PATH = 'E:/data/Ribonanza_RNA_folding'

In [2]:
import pandas as pd

test_data_pd = pd.read_csv(TEST_DATA_PATH)
test_data_pd.head()

,id_min,id_max,sequence_id,sequence,future
0,0,176,eee73c1836bc,GGGAACGACUCGAGUAGAGUCGAAAAUUUCCUUCCAAAUCCUGAGG...,0
1,177,353,d2a929af7a97,GGGAACGACUCGAGUAGAGUCGAAAAUGUAAUCAGAUUGCUUCUCC...,0
2,354,530,d39a4425ff45,GGGAACGACUCGAGUAGAGUCGAAAAAACACAUGAAUUUGAGGGUU...,0
3,531,707,1fc41e92d553,GGGAACGACUCGAGUAGAGUCGAAAAUCAGAGCUGGCAAAUGGAUG...,0
4,708,884,1d0826fb892f,GGGAACGACUCGAGUAGAGUCGAAAAUUUGGUAUUUGAUGCAUUAA...,0


In [3]:
import pandas as pd

test_extracted_pd = pd.read_csv(TEST_DATA_EXT_PATH)
test_extracted_pd.head()

,sequence,sequence_ext
0,GGGAACGACUCGAGUAGAGUCGAAAAUUUCCUUCCAAAUCCUGAGG...,(((((((((((.....)))))).....)))))......(((((((....
1,GGGAACGACUCGAGUAGAGUCGAAAAUGUAAUCAGAUUGCUUCUCC...,.....((((((.....))))))................((..((((...
2,GGGAACGACUCGAGUAGAGUCGAAAAAACACAUGAAUUUGAGGGUU...,.....((((((.....))))))...............((((((.(....
3,GGGAACGACUCGAGUAGAGUCGAAAAUCAGAGCUGGCAAAUGGAUG...,.....((((((.....))))))....((.(((..((((((.((.((...
4,GGGAACGACUCGAGUAGAGUCGAAAAUUUGGUAUUUGAUGCAUUAA...,.....((((((.....)))))).................((........


In [4]:
len(test_data_pd), len(test_extracted_pd)

(1343823, 1343823)

In [5]:
from tqdm import tqdm
import os

root_dir = BPP_DATA_PATH
prob_file_paths = []
for forder, _, files in tqdm(os.walk(root_dir)):
    for file in files:
        prob_file_paths.append(os.path.join(forder, file))

0it [00:00, ?it/s]

4369it [00:12, 352.78it/s]


In [6]:
unique_sequence_ids = set(test_data_pd['sequence_id'])

path_probs_dict = {}
count = 0

for i in tqdm(range(len(prob_file_paths))):
    file_name = os.path.splitext(os.path.basename(prob_file_paths[i]))[0]
    if file_name in unique_sequence_ids:
        path_probs_dict[file_name] = prob_file_paths[i]
        count += 1

count

100%|██████████| 2150396/2150396 [00:16<00:00, 132227.85it/s]


1381500

In [7]:
import sys
sys.path.append('..')

from python_scripts.transformers.dataset import RNADataset_probs_pred

rna_extracted_dataset = RNADataset_probs_pred(
    data=test_data_pd,
    data_ext=test_extracted_pd,
    path_probs=path_probs_dict,
    vocab=pd.read_csv(VOCAB_PATH),
    max_len=512
)

In [8]:
len(rna_extracted_dataset)

1343823

In [9]:
rna_extracted_dataset[0][0].shape

torch.Size([512])

In [10]:
from torchinfo import summary

import sys
sys.path.append('..')

from python_scripts.transformers.model import BERTCustomRNAReactivity, BERTCustom

bertmodel = BERTCustom(
    vocab_size=len(rna_extracted_dataset.vocab),
    hidden=512,
    dim_k=64,
    num_layer=12,
    num_attn_head=8
)
RNA_model = BERTCustomRNAReactivity(bertmodel)

summary(RNA_model)

Layer (type:depth-idx)                             Param #
BERTCustomRNAReactivity                            --
├─BERTCustom: 1-1                                  --
│    └─CombEmbedding: 2-1                          --
│    │    └─TokenEmbedding: 3-1                    11,776
│    │    └─PositionEmbedding: 3-2                 --
│    │    └─Dropout: 3-3                           --
│    └─ModuleList: 2-2                             --
│    │    └─EncoderBlock: 3-4                      3,152,385
│    │    └─EncoderBlock: 3-5                      3,152,385
│    │    └─EncoderBlock: 3-6                      3,152,385
│    │    └─EncoderBlock: 3-7                      3,152,385
│    │    └─EncoderBlock: 3-8                      3,152,385
│    │    └─EncoderBlock: 3-9                      3,152,385
│    │    └─EncoderBlock: 3-10                     3,152,385
│    │    └─EncoderBlock: 3-11                     3,152,385
│    │    └─EncoderBlock: 3-12                     3,152,385
│    │    

In [11]:
import torch

import lightning.pytorch as pl

import numpy as np

import sys
sys.path.append('..')

from python_scripts.transformers.dataset import RNADataModule
from python_scripts.transformers.task import RNATask

rna_datamodule = RNADataModule(predict_dataset=rna_extracted_dataset, batch_size=8, probs_adjusted=True)

# def rna_mae_loss(x: torch.tensor, y: torch.tensor, ignore_index=-100):
#     not_ignore = y != ignore_index
#     return torch.abs(x[not_ignore] - y[not_ignore]).mean()

# rna_optimizer = torch.optim.SGD(RNA_model.parameters(), 1e-3, 0.9)
# rna_scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer=rna_optimizer,
#     max_lr=1e-3,
#     steps_per_epoch=100,
#     epochs=5,
#     div_factor=1e2,
#     verbose=False
# )

rna_task = RNATask.load_from_checkpoint(
    checkpoint_path='./lightning_logs/BPP_8dim_231202/checkpoints/epoch=28-step=243339.ckpt',
    model=RNA_model,
    # loss_fn=rna_mae_loss,
    # optimizer=rna_optimizer,
    # scheduler=rna_scheduler,
)

trainer = pl.Trainer(precision='16-mixed')

predictions = trainer.predict(rna_task, datamodule=rna_datamodule)

predictions = np.concatenate(predictions, axis=1).clip(0, 1)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\TimJimTangtong\Miniconda3\envs\lightning\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\TimJimTangtong\Miniconda3\envs\lightning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Con

Predicting DataLoader 0:   0%|          | 0/167978 [00:00<?, ?it/s]

c:\Users\TimJimTangtong\Miniconda3\envs\lightning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:385: You have overridden `on_before_batch_transfer` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.


Predicting DataLoader 0: 100%|██████████| 167978/167978 [24:25:20<00:00,  1.91it/s]  


In [12]:
submission = pd.read_csv(SUBMISSION_SAMPLE_FILE_PATH)
submission['reactivity_2A3_MaP'] = predictions[0]
submission['reactivity_DMS_MaP'] = predictions[1]
submission.to_csv(SUBMISSION_FILE_PATH, index=False)

In [ ]:
# Check length generalization

import polars as pl
import matplotlib.pyplot as plt

df=pl.read_csv(SUBMISSION_FILE_PATH)
#some parameters
font_size=6
id1=269545321
id2=269724007
reshape1=391
reshape2=457
#get predictions
pred_DMS=df[id1:id2+1]['reactivity_DMS_MaP'].to_numpy().reshape(reshape1,reshape2)
pred_2A3=df[id1:id2+1]['reactivity_2A3_MaP'].to_numpy().reshape(reshape1,reshape2)
#plot mutate and map
fig = plt.figure()
plt.subplot(121)
plt.title(f'reactivity_DMS_MaP', fontsize=font_size)
plt.imshow(pred_DMS,vmin=0,vmax=1, cmap='gray_r')
plt.subplot(122)
plt.title(f'reactivity_2A3_MaP', fontsize=font_size)
plt.imshow(pred_2A3,vmin=0,vmax=1, cmap='gray_r')
plt.tight_layout()
plt.savefig(f"{DATA_PATH}/plot.png",dpi=500)
plt.clf()
plt.close()
